In [7]:
# Imports
import csv
import glob2
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import scanpy as sc
import squidpy as sq
import warnings
from anndata import AnnData
from scipy import sparse

In [8]:
# Globals
PRJ_DIR = "/scratch/gpfs/KANG/sereno/spatialstem"
SRC_DIR = f"{PRJ_DIR}/sourcefiles"
RAW_DIR = f"{SRC_DIR}/raw"
HAD_DIR = f"{SRC_DIR}/h5ad"
INT_DIR = f"{PRJ_DIR}/intermediates"
FIG_DIR = f"{PRJ_DIR}/figs"

In [128]:
# Debugs visium read-in function to accept DropletUtils-written hdf5s
from __future__ import annotations
import json
from pathlib import Path, PurePath
from typing import BinaryIO, Literal
import anndata.utils
import h5py
import numpy as np
import pandas as pd
from anndata import (
    AnnData,
    read_csv,
    read_excel,
    read_h5ad,
    read_hdf,
    read_loom,
    read_mtx,
    read_text,
)
from matplotlib.image import imread

def read_visium_debug(
    path: Path | str,
    genome: str | None = None,
    *,
    count_file: str = "filtered_feature_bc_matrix.h5",
    library_id: str | None = None,
    load_images: bool | None = True,
    source_image_path: Path | str | None = None,
) -> AnnData:
    """\
    Read 10x-Genomics-formatted visum dataset.

    In addition to reading regular 10x output,
    this looks for the `spatial` folder and loads images,
    coordinates and scale factors.
    Based on the `Space Ranger output docs`_.

    See :func:`~scanpy.pl.spatial` for a compatible plotting function.

    .. _Space Ranger output docs: https://support.10xgenomics.com/spatial-gene-expression/software/pipelines/latest/output/overview

    Parameters
    ----------
    path
        Path to directory for visium datafiles.
    genome
        Filter expression to genes within this genome.
    count_file
        Which file in the passed directory to use as the count file. Typically would be one of:
        'filtered_feature_bc_matrix.h5' or 'raw_feature_bc_matrix.h5'.
    library_id
        Identifier for the visium library. Can be modified when concatenating multiple adata objects.
    source_image_path
        Path to the high-resolution tissue image. Path will be included in
        `.uns["spatial"][library_id]["metadata"]["source_image_path"]`.

    Returns
    -------
    Annotated data matrix, where observations/cells are named by their
    barcode and variables/genes by gene name. Stores the following information:

    :attr:`~anndata.AnnData.X`
        The data matrix is stored
    :attr:`~anndata.AnnData.obs_names`
        Cell names
    :attr:`~anndata.AnnData.var_names`
        Gene names for a feature barcode matrix, probe names for a probe bc matrix
    :attr:`~anndata.AnnData.var`\\ `['gene_ids']`
        Gene IDs
    :attr:`~anndata.AnnData.var`\\ `['feature_types']`
        Feature types
    :attr:`~anndata.AnnData.obs`\\ `[filtered_barcodes]`
        filtered barcodes if present in the matrix
    :attr:`~anndata.AnnData.var`
        Any additional metadata present in /matrix/features is read in.
    :attr:`~anndata.AnnData.uns`\\ `['spatial']`
        Dict of spaceranger output files with 'library_id' as key
    :attr:`~anndata.AnnData.uns`\\ `['spatial'][library_id]['images']`
        Dict of images (`'hires'` and `'lowres'`)
    :attr:`~anndata.AnnData.uns`\\ `['spatial'][library_id]['scalefactors']`
        Scale factors for the spots
    :attr:`~anndata.AnnData.uns`\\ `['spatial'][library_id]['metadata']`
        Files metadata: 'chemistry_description', 'software_version', 'source_image_path'
    :attr:`~anndata.AnnData.obsm`\\ `['spatial']`
        Spatial spot coordinates, usable as `basis` by :func:`~scanpy.pl.embedding`.
    """
    path = Path(path)
    adata = sc.read_10x_h5(path / count_file, genome=genome)

    adata.uns["spatial"] = dict()

    from h5py import File

    with File(path / count_file, mode="r") as f:
        attrs = dict(f.attrs)
    if library_id is None:
        library_id = attrs["library_ids"]

    adata.uns["spatial"][library_id] = dict()

    if load_images:
        tissue_positions_file = (
            path / "spatial/tissue_positions.csv"
            if (path / "spatial/tissue_positions.csv").exists()
            else path / "spatial/tissue_positions_list.csv"
        )
        files = dict(
            tissue_positions_file=tissue_positions_file,
            scalefactors_json_file=path / "spatial/scalefactors_json.json",
            hires_image=path / "spatial/tissue_hires_image.png",
            lowres_image=path / "spatial/tissue_lowres_image.png",
        )

        # check if files exists, continue if images are missing
        for f in files.values():
            if not f.exists():
                if any(x in str(f) for x in ["hires_image", "lowres_image"]):
                    logg.warning(
                        f"You seem to be missing an image file.\n"
                        f"Could not find '{f}'."
                    )
                else:
                    raise OSError(f"Could not find '{f}'")

        adata.uns["spatial"][library_id]["images"] = dict()
        for res in ["hires", "lowres"]:
            try:
                adata.uns["spatial"][library_id]["images"][res] = imread(
                    str(files[f"{res}_image"])
                )
            except Exception:
                raise OSError(f"Could not find '{res}_image'")

        # read json scalefactors
        adata.uns["spatial"][library_id]["scalefactors"] = json.loads(
            files["scalefactors_json_file"].read_bytes()
        )

        adata.uns["spatial"][library_id]["metadata"] = {
            k: (str(attrs[k], "utf-8") if isinstance(attrs[k], bytes) else attrs[k])
            for k in ("chemistry_description", "software_version")
            if k in attrs
        }

        # read coordinates
        positions = pd.read_csv(
            files["tissue_positions_file"],
            header=0 if tissue_positions_file.name == "tissue_positions.csv" else None,
            index_col=0,
        )
        positions.columns = [
            "in_tissue",
            "array_row",
            "array_col",
            "pxl_col_in_fullres",
            "pxl_row_in_fullres",
        ]

        adata.obs = adata.obs.join(positions, how="left")

        adata.obsm["spatial"] = adata.obs[
            ["pxl_row_in_fullres", "pxl_col_in_fullres"]
        ].to_numpy()
        adata.obs.drop(
            columns=["pxl_row_in_fullres", "pxl_col_in_fullres"],
            inplace=True,
        )

        # put image path in uns
        if source_image_path is not None:
            # get an absolute path
            source_image_path = str(Path(source_image_path).resolve())
            adata.uns["spatial"][library_id]["metadata"]["source_image_path"] = str(
                source_image_path
            )

    return adata

In [24]:
# Dann: Mapping the developing human immune system across organs
dann_dir = f"{RAW_DIR}/pub01_dann"
dann_paths = glob2.glob(f"{dann_dir}/*")
# Already formatted...

In [53]:
# Niec: Lymphatics act as a signaling hub to regulate intestinal stem cell activity
niec_dir = f"{RAW_DIR}/pub02_niec"
niec_paths = glob2.glob(f"{niec_dir}/*")
niec_paths.sort()
niec_labels = ["largeintestine1", "largeintestine2", "smallintestine1", "smallintestine2"]
for niec_path, niec_label in zip(niec_paths, niec_labels):
    # Will warn you that your variable names aren't unique, fixed below.
    with warnings.catch_warnings(action="ignore"):
        visium_in = sc.read_visium(niec_path)
    visium_in.var_names_make_unique()
    h5_out_path = f"{HAD_DIR}/p02_{niec_label}.h5ad"
    visium_in.write_h5ad(h5_out_path)
    print(f"Written: {h5_out_path}")

Written: /scratch/gpfs/KANG/sereno/spatialstem/sourcefiles/h5ad/p2_largeintestine1.h5ad
Written: /scratch/gpfs/KANG/sereno/spatialstem/sourcefiles/h5ad/p2_largeintestine2.h5ad
Written: /scratch/gpfs/KANG/sereno/spatialstem/sourcefiles/h5ad/p2_smallintestine1.h5ad
Written: /scratch/gpfs/KANG/sereno/spatialstem/sourcefiles/h5ad/p2_smallintestine2.h5ad


In [49]:
# Calvanese: Mapping human hematopoietic stem cells from hemogenic endothelium to birth
calv_dir = f"{RAW_DIR}/pub03_calvanese"
calv_paths = glob2.glob(f"{calv_dir}/*")
calv_paths.sort()
for idx, calv_path in enumerate(calv_paths):
    with warnings.catch_warnings(action="ignore"):
        visium_in = sc.read_visium(calv_path)
    visium_in.var_names_make_unique()
    h5_out_path = f"{HAD_DIR}/p03_hsc{idx+1}.h5ad"
    visium_in.write_h5ad(h5_out_path)
    print(f"Written: {h5_out_path}")

['/scratch/gpfs/KANG/sereno/spatialstem/sourcefiles/raw/pub3_calvanese/HM-1', '/scratch/gpfs/KANG/sereno/spatialstem/sourcefiles/raw/pub3_calvanese/HM-2', '/scratch/gpfs/KANG/sereno/spatialstem/sourcefiles/raw/pub3_calvanese/HM-4', '/scratch/gpfs/KANG/sereno/spatialstem/sourcefiles/raw/pub3_calvanese/HM-5', '/scratch/gpfs/KANG/sereno/spatialstem/sourcefiles/raw/pub3_calvanese/HM-6', '/scratch/gpfs/KANG/sereno/spatialstem/sourcefiles/raw/pub3_calvanese/HM-7', '/scratch/gpfs/KANG/sereno/spatialstem/sourcefiles/raw/pub3_calvanese/HM-8']


In [90]:
# Larouche: Spatiotemporal mapping of immune and stem cell dysregulation after volumetric muscle loss
laro_dir = f"{RAW_DIR}/pub04_larouche"
laro_paths_raw = glob2.glob(f"{laro_dir}/*")
# Filters out raw RDS objects
laro_paths = [path for path in laro_paths_raw if 'GSE205707' not in path]
laro_paths.sort()
for laro_path in laro_paths:
    laro_lab = laro_path.split("/")[-1]
    visium_in = sc.read_10x_mtx(laro_path)
    coords_in = np.genfromtxt(f"{laro_path}/coords.csv", delimiter=",", dtype="int64")
    visium_in.obsm["spatial"] = coords_in
    h5_out_path = f"{HAD_DIR}/p04_{laro_lab}.h5ad"
    visium_in.write_h5ad(h5_out_path)
    print(f"Written: {h5_out_path}")

Written: /scratch/gpfs/KANG/sereno/spatialstem/sourcefiles/h5ad/p4_caD0IR1.h5ad
Written: /scratch/gpfs/KANG/sereno/spatialstem/sourcefiles/h5ad/p4_caD0IR2.h5ad
Written: /scratch/gpfs/KANG/sereno/spatialstem/sourcefiles/h5ad/p4_caD14E1.h5ad
Written: /scratch/gpfs/KANG/sereno/spatialstem/sourcefiles/h5ad/p4_caD14M1.h5ad
Written: /scratch/gpfs/KANG/sereno/spatialstem/sourcefiles/h5ad/p4_caD14M2.h5ad
Written: /scratch/gpfs/KANG/sereno/spatialstem/sourcefiles/h5ad/p4_caD7E1.h5ad
Written: /scratch/gpfs/KANG/sereno/spatialstem/sourcefiles/h5ad/p4_caD7M1.h5ad
Written: /scratch/gpfs/KANG/sereno/spatialstem/sourcefiles/h5ad/p4_caD7M2.h5ad
Written: /scratch/gpfs/KANG/sereno/spatialstem/sourcefiles/h5ad/p4_mmD141.h5ad
Written: /scratch/gpfs/KANG/sereno/spatialstem/sourcefiles/h5ad/p4_mmD142.h5ad
Written: /scratch/gpfs/KANG/sereno/spatialstem/sourcefiles/h5ad/p4_mmITD1.h5ad
Written: /scratch/gpfs/KANG/sereno/spatialstem/sourcefiles/h5ad/p4_mmITD2.h5ad
Written: /scratch/gpfs/KANG/sereno/spatialstem/

In [5]:
# # Error in this dataset (Shani, P5), deprecated for now.
# # Shani: The spatio-temporal program of liver zonal regeneration
# shan_dir = f"{RAW_DIR}/pub05_shani"
# shan_paths = glob2.glob(f"{shan_dir}/*.h5")
# shan_paths.sort()
# print(shan_paths)
# # Read metadata to get visium coords
# visium_meta_dict = {}
# with open(f'{shan_dir}/Visium_Meta_data.txt', mode ='r')as meta_file:
#     shan_meta = csv.reader(meta_file)
#     next(shan_meta)
#     for line in shan_meta:
#         sample = line[1].replace("t_", "")
#         x_coord = np.int64(line[4])
#         y_coord = np.int64(line[5])
#         coords = [x_coord, y_coord]
#         if sample not in visium_meta_dict:
#             visium_meta_dict[sample] = []
#         visium_meta_dict[sample].append(coords)
# # Notice that this does not work due to the visium_in not lining up with metadata.
# for shan_path in shan_paths:
#     # Extract mouse id
#     shan_lab = shan_path.split("/")[-1].replace("Visium_", "").replace("_raw_feature_bc_matrix.h5", "")
#     # Will warn you that your variable names aren't unique, fixed below.
#     with warnings.catch_warnings(action="ignore"):
#         visium_in = sc.read_10x_h5(shan_path)
#     visium_in.var_names_make_unique()
#     coords = np.asarray(visium_meta_dict[shan_lab])
#     visium_in.obsm["spatial"] = coords
#     h5_out_path = f"{HAD_DIR}/p05_{shan_lab}.h5ad"
#     # visium_in.write_h5ad(h5_out_path)

['/scratch/gpfs/KANG/sereno/spatialstem/sourcefiles/raw/pub5_shani/Visium_24h_m1_raw_feature_bc_matrix.h5', '/scratch/gpfs/KANG/sereno/spatialstem/sourcefiles/raw/pub5_shani/Visium_24h_m2_raw_feature_bc_matrix.h5', '/scratch/gpfs/KANG/sereno/spatialstem/sourcefiles/raw/pub5_shani/Visium_48h_m4_raw_feature_bc_matrix.h5', '/scratch/gpfs/KANG/sereno/spatialstem/sourcefiles/raw/pub5_shani/Visium_48h_m5_raw_feature_bc_matrix.h5', '/scratch/gpfs/KANG/sereno/spatialstem/sourcefiles/raw/pub5_shani/Visium_72h_m1_raw_feature_bc_matrix.h5', '/scratch/gpfs/KANG/sereno/spatialstem/sourcefiles/raw/pub5_shani/Visium_72h_m2_raw_feature_bc_matrix.h5']


In [40]:
# Some example Shani objects: obj 1
shan_path = shan_paths[1]
shan_lab = shan_path.split("/")[-1].replace("Visium_", "").replace("_raw_feature_bc_matrix.h5", "")
# Will warn you that your variable names aren't unique, fixed below.
with warnings.catch_warnings(action="ignore"):
    visium_in = sc.read_10x_h5(shan_path)
visium_in.var_names_make_unique()
print(shan_path)
visium_in

/scratch/gpfs/KANG/sereno/spatialstem/sourcefiles/raw/pub5_shani/Visium_24h_m2_raw_feature_bc_matrix.h5


AnnData object with n_obs × n_vars = 4992 × 32285
    var: 'gene_ids', 'feature_types', 'genome'

In [41]:
# Some example Shani objects: obj 2
shan_path = shan_paths[2]
shan_lab = shan_path.split("/")[-1].replace("Visium_", "").replace("_raw_feature_bc_matrix.h5", "")
# Will warn you that your variable names aren't unique, fixed below.
with warnings.catch_warnings(action="ignore"):
    visium_in = sc.read_10x_h5(shan_path)
visium_in.var_names_make_unique()
print(shan_path)
visium_in

/scratch/gpfs/KANG/sereno/spatialstem/sourcefiles/raw/pub5_shani/Visium_48h_m4_raw_feature_bc_matrix.h5


AnnData object with n_obs × n_vars = 4991 × 32285
    var: 'gene_ids', 'feature_types', 'genome'

In [3]:
# Biermann: Dissecting the treatment-naive ecosystem of human melanoma brain metastasis
# Index of count files and spatial info should be equivalent.
bier_dir = f"{RAW_DIR}/pub06_biermann"
bier_count_paths = glob2.glob(f"{bier_dir}/*raw_counts*")
bier_count_paths.sort()
bier_coord_paths = glob2.glob(f"{bier_dir}/*spatial_info*")
bier_coord_paths.sort()
for bier_count_path, bier_coord_path in zip(bier_count_paths, bier_coord_paths):
    counts = pd.read_csv(bier_count_path, compression='gzip', index_col=0).T
    coords = pd.read_csv(bier_coord_path, compression='gzip', usecols=['xcoord', 'ycoord'])
    bier_path_split = bier_count_path.split("/")[-1].split("_")
    sample_id = ("_").join(bier_path_split[1:]).replace("_slide_raw_counts.csv.gz", "")
    bier_ad = sc.AnnData(sparse.csr_matrix(counts), counts.index.to_frame(), counts.columns.to_frame())
    # Manual coordinate frame build
    coords_arr = np.asarray(coords)
    bier_ad.obsm["spatial"] = coords_arr
    # Manual feature info frame build.
    gene_names = counts.columns
    feature_type_rep = ["Gene Expression"] * len(gene_names)
    bier_gene_frame = pd.DataFrame(index=gene_names, data={"gene_ids": gene_names, "feature_types": feature_type_rep})
    bier_ad.var = bier_gene_frame
    # Manual obs frame build
    barcodes = counts.index
    bier_obs_frame = pd.DataFrame(index=barcodes, data={"barcode": barcodes})
    bier_obs_frame.index.name = "barcode"
    bier_ad.obs = bier_obs_frame
    h5_out_path = f"{HAD_DIR}/0p6_{sample_id}.h5ad"
    bier_ad.write_h5ad(h5_out_path)
    print(f"Written: {h5_out_path}")

Written: /scratch/gpfs/KANG/sereno/spatialstem/sourcefiles/h5ad/p6_MBM05_rep1.h5ad
Written: /scratch/gpfs/KANG/sereno/spatialstem/sourcefiles/h5ad/p6_MBM05_rep2.h5ad
Written: /scratch/gpfs/KANG/sereno/spatialstem/sourcefiles/h5ad/p6_MBM05_rep3.h5ad
Written: /scratch/gpfs/KANG/sereno/spatialstem/sourcefiles/h5ad/p6_MBM06.h5ad
Written: /scratch/gpfs/KANG/sereno/spatialstem/sourcefiles/h5ad/p6_MBM07.h5ad
Written: /scratch/gpfs/KANG/sereno/spatialstem/sourcefiles/h5ad/p6_MBM08.h5ad
Written: /scratch/gpfs/KANG/sereno/spatialstem/sourcefiles/h5ad/p6_MBM11_rep1.h5ad
Written: /scratch/gpfs/KANG/sereno/spatialstem/sourcefiles/h5ad/p6_MBM11_rep2.h5ad
Written: /scratch/gpfs/KANG/sereno/spatialstem/sourcefiles/h5ad/p6_MBM11_rep3.h5ad
Written: /scratch/gpfs/KANG/sereno/spatialstem/sourcefiles/h5ad/p6_MBM13.h5ad
Written: /scratch/gpfs/KANG/sereno/spatialstem/sourcefiles/h5ad/p6_MBM18.h5ad
Written: /scratch/gpfs/KANG/sereno/spatialstem/sourcefiles/h5ad/p6_ECM01_rep1.h5ad
Written: /scratch/gpfs/KANG/s

In [6]:
# Rajachandran: Dissecting the spermatogonial stem cell niche using spatial transcriptomics
raja_dir = f"{RAW_DIR}/pub07_rajachandran"
raja_paths = glob2.glob(f"{raja_dir}/*")
raja_paths.sort()
for raja_path in raja_paths:
    raja_counts = glob2.glob(f"{raja_path}/*MappedDGE*")[0]
    raja_coords = glob2.glob(f"{raja_path}/*BeadLocations*")[0]
    # Shitty hack for inputs that contain bad data columns -- optimize later.
    try:
        counts = pd.read_csv(raja_counts, index_col=1)
        counts.drop("Unnamed: 0", axis=1, inplace=True)
    except KeyError:
        counts = pd.read_csv(raja_counts, index_col=0)
    coords = pd.read_csv(raja_coords, usecols=['x', 'y'])
    sample_id = raja_path.replace(f"{raja_dir}/", "")
    raja_ad = sc.AnnData(sparse.csr_matrix(counts), counts.index.to_frame(), counts.columns.to_frame())
    # Manual coordinate frame build
    coords_arr = np.asarray(coords)
    raja_ad.obsm["spatial"] = coords_arr
    # Manual feature info frame build.
    gene_names = counts.columns
    feature_type_rep = ["Gene Expression"] * len(gene_names)
    raja_gene_frame = pd.DataFrame(index=gene_names, data={"gene_ids": gene_names, "feature_types": feature_type_rep})
    raja_ad.var = raja_gene_frame
    h5_out_path = f"{HAD_DIR}/p7_{sample_id}.h5ad"
    raja_ad.write_h5ad(h5_out_path)
    print(f"Written: {h5_out_path}")

Written: /scratch/gpfs/KANG/sereno/spatialstem/sourcefiles/h5ad/p7_diabetes3.h5ad
Written: /scratch/gpfs/KANG/sereno/spatialstem/sourcefiles/h5ad/p7_diabetes1.h5ad
Written: /scratch/gpfs/KANG/sereno/spatialstem/sourcefiles/h5ad/p7_diabetes2.h5ad
Written: /scratch/gpfs/KANG/sereno/spatialstem/sourcefiles/h5ad/p7_wtdiab2.h5ad
Written: /scratch/gpfs/KANG/sereno/spatialstem/sourcefiles/h5ad/p7_wtdiab3.h5ad
Written: /scratch/gpfs/KANG/sereno/spatialstem/sourcefiles/h5ad/p7_wtdiab1.h5ad


In [72]:
# Chen: Human neural stem cells restore spatial memory in a transgenic Alzheimer's disease mouse model 
# by an immunomodulating mechanism
# Need to write hdf5s in R before running.
chen_dir = f"{RAW_DIR}/pub8_chen"
chen_paths = glob2.glob(f"{chen_dir}/*")
chen_paths.sort()
for chen_path in chen_paths:
    sample_id = chen_path.split("/")[-1].replace("Sample_3792-FM-", "")
    chen_ad = read_visium_debug(chen_path)
    h5_out_path = f"{HAD_DIR}/p8_{sample_id}.h5ad"
    chen_ad.write_h5ad(h5_out_path)
    print(f"Written: {h5_out_path}")

In [32]:
# Chaker: Pregnancy-responsive pools of adult neural stem cells for transient neurogenesis in mothers
# Note: was missing tissue lowres image, copied hires image to it.
chak_dir = f"{RAW_DIR}/pub09_chaker"
chak_paths = glob2.glob(f"{chak_dir}/*")
chak_paths.sort()
for chak_path in chak_paths:
    chak_path_split = chak_path.split("/")[-1].split("_")
    sample_id = "_".join(chak_path_split[1:])
    sample_id
    with warnings.catch_warnings(action="ignore"):
        chak_ad = sc.read_visium(chak_path)
    h5_out_path = f"{HAD_DIR}/p09_{sample_id}.h5ad"
    chak_ad.write_h5ad(h5_out_path)
    print(f"Written: {h5_out_path}")

Written: /scratch/gpfs/KANG/sereno/spatialstem/sourcefiles/h5ad/p09_A2_OB_virgin.h5ad
Written: /scratch/gpfs/KANG/sereno/spatialstem/sourcefiles/h5ad/p09_B2_OB_mother.h5ad
Written: /scratch/gpfs/KANG/sereno/spatialstem/sourcefiles/h5ad/p09_C2_OB_virgin.h5ad
Written: /scratch/gpfs/KANG/sereno/spatialstem/sourcefiles/h5ad/p09_D2_OB_mother.h5ad


In [ ]:
# # Error in this dataset (Cohen, P10), no spatial info (positions.tsv) in uploaded data.
# Cohen: Regulatory T cells in skin mediate immune privilege of the hair follicle stem cell niche